## Bài tập tính điểm cộng lần 2: Đơn hình theo phương pháp big-M

- MSSV: 22120222
- Họ và tên: Võ Văn Nam
- Lớp: CQ2022/5

**Nội dung bài tập**

SV tham khảo yêu cầu đề bài cụ thể trong slide 3: Cho hệ ràng buộc gồm m điều kiện (với m = 2, 3 hoặc 4) của bài toán QHTT chính tắc n <= 10 biến cùng hệ số của hàm mục tiêu dạng max/min tất cả các số nhập vào đều là số nguyên có trị tuyệt đối không vượt quá 10.

Yêu cầu: Hãy in ra bảng đơn hình xuất phát theo phương pháp big-M với số biến bổ sung là ít nhất, trong đó chọn: M=1000.

In [8]:
'''
Cho hệ ràng buộc gồm m điều kiện (với m = 2, 3 hoặc 4) của bài toán QHTT chính tắc n <= 10 biến 
cùng hệ số của hàm mục tiêu dạng max/min tất cả các số nhập vào đều là số nguyên có trị tuyệt đối không vượt quá 10.
Yêu cầu: Hãy in ra bảng đơn hình xuất phát theo phương pháp big-M với số biến bổ sung là ít nhất, trong đó chọn: M=1000.

Input: 
1 2 3 // hệ số của hàm mục tiêu
1 -1 2 = 3 (dấu này có thể là >=, <= hoặc =).
3 4 -2 = 5
Output: 
x4  -1000   3   1   -1  2   1   0
x5  -1000   5   3   4   -1  0   1
        -8000   -4001   -3002   -3   0   0
'''

M = 1000  # Giá trị M để sự dụng phương pháp big-M

def parse_constraint(line, n):
    """
    Hàm phân tích một dòng ràng buộc.
    Ràng buộc có dạng: a1 a2 ... an quan_hệ b
    Quan hệ có thể là '=', '<=' hoặc '>='.
    Trả về: (list các hệ số của biến gốc, quan hệ, b)
    """
    tokens = line.strip().split()
    coef = list(map(int, tokens[:n])) # Hệ số của biến gốc
    rel = tokens[n] # Quan hệ
    b_val = int(tokens[n+1]) 
    return coef, rel, b_val

def process_input_data(input_data):
    """
    Hàm xử lý input_data, lấy các hệ số của hàm mục tiêu.
    Tạo danh sách các ràng buộc với thông tin hệ số, quan hệ và hằng số.
    Trả về: (obj_coeffs, constraints)
    """
    lines = [l for l in input_data if l.strip() != '']
    
    # Dòng đầu tiên: hệ số của hàm mục tiêu (cho các biến x1, ..., xn)
    obj_coeffs = list(map(int, lines[0].strip().split()))
    n = len(obj_coeffs)
    
    constraints = []
    for line in lines[1:]:
        coef, rel, b_val = parse_constraint(line, n)
        constraints.append({
            'coef': coef,
            'rel': rel,
            'b': b_val
        })
    return obj_coeffs, constraints

def build_tableau(obj_coeffs, constraints, M):
    """
    Xây dựng bảng đơn hình theo phương pháp Big-M
    Trả về: (tableau_rows, objective_row)
       - tableau_rows: danh sách các hàng của bảng đơn hình (bao gồm thông tin biến cơ sở).
       - objective_row: hàng mục tiêu đã được xây dựng.
    """
    n = len(obj_coeffs)
    m = len(constraints)
    
    # Danh sách các biến bổ sung (extra variables)
    # Mỗi phần tử gồm i_cons: index của ràng buộc, type: loại biến, c: hệ số trong hàm mục tiêu
    extra_vars = []
    # Với mỗi hàng ràng buộc, lưu thông tin về biến cơ sở (nếu có)
    basis = [None] * m
    
    # Danh sách chứa vector hệ số của biến bổ sung cho từng hàng
    row_extra = []
    for i, i_cons in enumerate(constraints):
        current_index = len(extra_vars)
        # Thêm biến bổ sung theo loại quan hệ:
        if i_cons['rel'] == '<=':
            extra_vars.append({'i_cons': i, 'type': '1', 'c': 0}) # Biến bổ sung +x bù vào <=
            basis[i] = current_index
        elif i_cons['rel'] == '=':
            extra_vars.append({'i_cons': i, 'type': '0', 'c': -M}) 
            basis[i] = current_index
        elif i_cons['rel'] == '>=':
            extra_vars.append({'i_cons': i, 'type': '-1', 'c': 0}) # Biến -x bù vào >=
            extra_vars.append({'i_cons': i, 'type': '0', 'c': -M}) # Biến +x bù vào = do biến bên trên mang dấu -
            basis[i] = current_index + 1
        else:
            print(f"Quan hệ {i_cons['rel']} không hợp lệ")
            exit(1)
        
        # Xây dựng vector hệ số cho biến bổ sung của hàng i với độ dài hiện tại = số biến bổ sung hiện có
        row_vec = [0] * len(extra_vars)
        # Với từng biến bổ sung liên quan đến hàng i, thiết lập hệ số:
        # 1 đối với biến <= và =, -1 đối với biến >=
        for j in range(len(extra_vars)):
            if extra_vars[j]['i_cons'] == i:
                if extra_vars[j]['type'] in ['1', '0']:
                    row_vec[j] = 1
                elif extra_vars[j]['type'] == '-1':
                    row_vec[j] = -1
        row_extra.append(row_vec)
    
    # Cập nhật lại các hàng trong row_extra sao cho độ dài của vector hệ số biến bổ sung bằng total_extra
    # Biến bổ sung nào không có sẽ có hệ số = 0
    total_extra = len(extra_vars)
    for i in range(len(row_extra)):
        if len(row_extra[i]) < total_extra:
            row_extra[i].extend([0] * (total_extra - len(row_extra[i])))
    
    # Xây dựng bảng đơn hình: mỗi hàng gồm:
    # [Tên biến cơ sở, HS, PA, hệ số của biến gốc, hệ số của các biến bổ sung]
    tableau_rows = []
    for i, i_cons in enumerate(constraints):
        idx = basis[i]
        HS = extra_vars[idx]['c'] if idx is not None else 0 # Hệ số
        row = [HS, i_cons['b']] + i_cons['coef'] + row_extra[i]
        # Đặt tên biến: các biến gốc là x1,...,xn; nên các biến bổ sung được đánh số từ x(n+1) trở đi.
        var_index = n + idx + 1 if idx is not None else None
        tableau_rows.append((f"x{var_index}" if var_index is not None else "", row))
    
    # Tính hàng mục tiêu theo công thức đơn hình với big-M
    const_term = 0
    # Tổng số cột bao gồm biến gốc (n cột) và biến bổ sung (total_extra cột)
    num_cols = n + total_extra
    reduced = [0] * num_cols
    for i, i_cons in enumerate(constraints):
        HS = tableau_rows[i][1][0]  # Hệ số
        PA = tableau_rows[i][1][1] # Phương án
        const_term += HS * PA
        # Lấy ra vector hệ số biến gốc và biến bổ sung của hàng i
        row_data = tableau_rows[i][1][2:]
        for j in range(num_cols):
            reduced[j] += HS * row_data[j]
    
    # Trừ đi hệ số của hàm mục tiêu ban đầu của biến gốc
    for j in range(n):
        reduced[j] -= obj_coeffs[j]
    # Trừ hệ số của hàm mục tiêu của biến bổ sung theo loại quan hệ
    for j in range(n, num_cols):
        extra_cost = extra_vars[j - n]['c']
        reduced[j] -= extra_cost

    objective_row = [const_term] + reduced

    return tableau_rows, objective_row

def print_tableau(tableau_rows, objective_row):
    """
    Hàm in ra bảng đơn hình.
    tableau_rows: danh sách các hàng của bảng đơn hình.
    objective_row: hàng mục tiêu đã được xây dựng.
    """
    for label, row in tableau_rows:
        row_str = " ".join(str(x) for x in row)
        print(f"{label} {row_str}".strip())
    obj_str = " ".join(str(x) for x in objective_row)
    print(" " + obj_str)

def main():
    # Dữ liệu đầu vào, dòng đầu tiên là hệ số của hàm mục tiêu, các dòng tiếp theo là các ràng buộc
    input_data = [
        "1 2 3",       # Hệ số của hàm mục tiêu
        "1 -1 2 = 3",  # Ràng buộc thứ 1
        "3 4 -2 = 5"   # Ràng buộc thứ 2
    ]
    
    # Xử lý input
    obj_coeffs, constraints = process_input_data(input_data)
    
    # Xây dựng bảng đơn hình theo phương pháp big-M
    tableau_rows, objective_row = build_tableau(obj_coeffs, constraints, M)
    
    # In bảng đơn hình
    print_tableau(tableau_rows, objective_row)
    
if __name__ == '__main__':
    main()


x4 -1000 3 1 -1 2 1 0
x5 -1000 5 3 4 -2 0 1
 -8000 -4001 -3002 -3 0 0
